LLM에서 도구를 사용하는 가장 신뢰할 수 있는 방법 중 하나는 도구 호출 API(때로는 함수 호출이라고도 함)를 사용하는 것입니다. 이는 도구 호출을 명시적으로 지원하는 모델에서만 작동합니다. 도구 호출을 지원하는 모델을 여기에서 확인할 수 있으며, 이 가이드 에서 도구 호출을 사용하는 방법에 대해 자세히 알아볼 수 있습니다 .

In [1]:
%pip install --upgrade --quiet langchain langchain-openai langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 962.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
from uuid import uuid4

# unique_id를 uuid4()로 생성
unique_id = str(uuid4())

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain Tool Use Quickstart - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [4]:
unique_id

'af2709d0-1528-49b3-b960-7cbfbefa6320'

In [5]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [7]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

### Tool/function calling

LLM에서 도구를 사용하는 가장 신뢰할 수 있는 방법 중 하나는 도구 호출 API(때로는 함수 호출이라고도 함)를 사용하는 것입니다. 이는 도구 호출을 명시적으로 지원하는 모델에서만 작동합니다.

- 도구 호출을 지원하는 모델을  https://python.langchain.com/v0.1/docs/integrations/chat/ 에서 확인할 수 있으며,
- https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/ 에서 도구 호출을 사용하는 방법에 대해 자세히 알아볼 수 있습니다 .

먼저 모델과 도구를 정의하겠습니다. multiply 단일 도구로 시작하겠습니다.


In [8]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd"

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

bind_tools모델에 대한 각 호출의 일부로 도구 정의를 전달하여 모델이 적절한 경우 도구를 호출할 수 있도록 합니다 .

In [10]:
llm_with_tools = llm.bind_tools([multiply])

모델이 도구를 호출하면 이는 출력의 AIMessage.tool_calls 속성 에 표시됩니다 .

In [11]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_wlb75OiLJf54IPCyQ9P9EBsp',
  'type': 'tool_call'}]

좋습니다! 도구 호출을 생성할 수 있습니다. 하지만 실제로 도구를 호출하고 싶다면 어떨까요? 그렇게 하려면 생성된 도구 인수를 도구에 전달해야 합니다. 간단한 예로 첫 번째 tool_call의 인수를 추출해 보겠습니다.
- llm_with_tools: 자연어 입력을 처리하여 도구 호출을 결정합니다.
- 익명 함수는 LLM의 도구 호출 결과에서 인자를 가져옵니다.
- 곱셈 함수가 해당 인자(4와 23)를 사용하여 곱셈 연산을 수행합니다.
- 이 모든 단계가 체인으로 연결되어 있어, "What's four times 23" 같은 질문에 대해 최종적으로 92라는 결과가 도출됩니다.

In [12]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

92

### Agents

체인은 사용자 입력에 필요한 특정 도구 사용 순서를 알고 있을 때 유용합니다. 하지만 특정 사용 사례의 경우 도구를 사용하는 횟수는 입력에 따라 달라집니다. 이런 경우 모델 자체가 도구를 사용할 횟수와 순서를 결정하도록 합니다. 에이전트는 우리가 바로 이를 할 수 있도록 합니다.

LangChain에는 다양한 사용 사례에 최적화된 여러 내장 에이전트가 제공됩니다. 여기에서 모든 에이전트 유형 에 대해 읽어보세요 .

우리는 일반적으로 가장 신뢰할 수 있고 대부분의 사용 사례에 권장되는 도구인 호출 에이전트를 사용할 것입니다 .

In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [14]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [15]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [16]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The final result of taking 3 to the fifth power, multiplying it by the sum of twelve and three, and then squaring the whole result is **13,286,025**.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The final result of taking 3 to the fifth power, multiplying it by the sum of twelve and three, and then squaring the whole result is **13,286,025**.'}